# 第7章 现代卷积神经网络

## 7.1 深度卷积神经网络(AlexNet) 

### 练习 7.1.1 

试着增加迭代轮数。对比LeNet的结果有什么不同？为什么？

**解答：** 

batch_size:128, lr;0.01, epochs:10
loss 0.326, train acc 0.881, test acc 0.881
1626.4 examples/sec on cuda:0
![7_1_1_1.png](../.././images/7_1_1_1.png)
batch_size:128, lr;0.01, epochs:20
loss 0.247, train acc 0.909, test acc 0.902
1641.9 examples/sec on cuda:0
![7_1_1_2.png](../.././images/7_1_1_2.png)

**迭代次数越多训练集的损失值和准确度越高，但测试集的准确率没有训练集的高。迭代次数过多容易发生过拟合现象。**

### 练习 7.1.2

AlexNet对Fashion-MNIST数据集来说可能太复杂了。
1. 尝试简化模型以加快训练速度，同时确保准确性不会显著下降。
1. 设计一个更好的模型，可以直接在$28 \times 28$图像上工作。

**解答：** 

net28更适合在28×28的图像上工作，相比原始网络，net28训练速度更快，loss值更低，准确率更高。降低kernel_size的大小和使用更少的全连接神经元数量，降低网络的参数数量。

In [6]:
import torch
import torch.nn as nn


net28 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=5, stride=2, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=1),

    nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(128, 128, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(128, 96, kernel_size=3, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),

    nn.Linear(96 * 5 * 5, 2048), nn.ReLU(),
    nn.Dropout(p=0.5), 
    nn.Linear(2048, 1024), nn.ReLU(),
    nn.Dropout(p=0.5),

    nn.Linear(1024, 10)
)

batch_size = 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
lr, num_epochs = 0.02, 10
d2l.train_ch6(net28, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

### 练习 7.1.3

修改批量大小，并观察模型精度和GPU显存变化。

**解答：** 

更改batch_size为64，模型精度略微上升，GPU显存使用降低

batch_size:64, lr;0.01, epochs:10

loss 0.275, train acc 0.899, test acc 0.897
1326.2 examples/sec on cuda:0

![7_1_3.png](../.././images/7_1_3_1.png)

### 练习 7.1.4

分析了AlexNet的计算性能。
1. 在AlexNet中主要是哪部分占用显存？
1. 在AlexNet中主要是哪部分需要更多的计算？
1. 计算结果时显存带宽如何？

**解答：** 

- 在AlexNet中，占据最多显存的层是第一层卷积层和第二层卷积层。这是因为卷积层需要存储卷积核参数和中间计算结果，而且随着层数的增加，卷积层的输出通道数也会逐步增加，导致占用的显存也随之增加。相比之下，池化层和ReLU激活函数层等计算量较小的层，占用的显存相对较少。全连接层的显存占用量也较大，但是整个模型中只有两层全连接层，因此相对于卷积层而言，对显存的占用不是很明显。

- 在这个模型中，需要更多计算的层主要是全连接层。全连接层的计算量与输入和输出的维度相关，因此输入维度为6400的第一层全连接层和输出维度为4096的第二层全连接层需要更多的计算。这两个全连接层的计算量是其他卷积层的数倍，也是整个模型计算量的瓶颈。因此，在实际应用中，可以考虑减少全连接层的数量和神经元个数，以减少计算量并提高模型的运行速度。

- 全连接层的计算也需要大量的数据传输，因为全连接层的输入和输出都是向量形式，需要在计算过程中频繁地读写显存。



### 练习 7.1.5

将dropout和ReLU应用于LeNet-5，效果有提升吗？再试试预处理会怎么样？

**解答：** 

将dropout和ReLU应用于LeNet-5可能会提升其性能：

- Dropout可以防止过拟合，避免神经元之间的过度共适应，从而提高网络的泛化能力。

- ReLU可以增加网络的非线性，从而提高其表示能力。 使用这些技术的LeNet-5可能会表现得更好，但具体效果取决于具体的数据集和超参数设置。 预处理也可能对LeNet-5的性能产生影响。一些可能的预处理方法包括：

- 归一化数据：将输入数据缩放到0到1的范围内，可以帮助网络更快地收敛并提高其泛化能力。

- 数据增强：通过对输入数据进行旋转、平移、翻转等操作，可以增加训练数据的多样性，从而提高网络的泛化能力。

- 特征提取：使用特征提取方法如PCA、LDA、ICA等，可以减少输入数据的维度并提取最有用的特征，从而帮助网络更好地学习。 这些预处理方法可能会对LeNet-5的性能产生积极影响，但具体效果也取决于数据集和超参数设置。



## 7.2 使用块的网络（VGG） 

### 练习 7.2.1 

打印层的尺寸时，我们只看到8个结果，而不是11个结果。剩余的3层信息去哪了？

**解答：** 

打印出来有5个Sequential层和3个Linear层，是因为后三个Sequential层中，每层有两个卷积块。



### 练习 7.2.2

与AlexNet相比，VGG的计算要慢得多，而且它还需要更多的显存。分析出现这种情况的原因。

**解答：** 

1. 更深的网络结构：VGG相比于AlexNet增加了网络层数，采用了16-19层的卷积层，这导致了计算量的增加和显存的需求增加。

2. 更小的卷积核：VGG网络中采用了较小的3x3卷积核，相比AlexNet的7x7卷积核，这样做的好处是可以增加网络的深度，但是却会导致计算量和显存的需求增加。因为采用了小卷积核，需要进行更多的卷积操作，这增加了计算量；同时，每个卷积核的参数数量更多，需要更多的显存进行存储，增加了显存需求。 因此，VGG相比于AlexNet具有更深的网络结构和更小的卷积核，这使得它需要更多的计算和显存。



### 练习 7.2.3

尝试将Fashion-MNIST数据集图像的高度和宽度从224改为96。这对实验有什么影响？

**解答：** 

1. 准确率下降：由于VGG网络是在ImageNet数据集上进行训练的，其输入图像大小为224x224。将输入图像大小从224x224改为96x96，会导致图像信息的丢失，从而影响网络的准确率。

2. 训练时间缩短：由于输入图像大小变小，计算量也相应减小，这会使得模型的训练时间缩短。

3. 内存占用减少：由于输入图像大小变小，模型参数数量也会减少，这将使得模型所需要的内存占用减少。 综上所述，将Fashion-MNIST数据集图像的高度和宽度从224改为96，可能会导致模型准确率下降，但同时也会缩短训练时间，减少内存占用。因此，需要根据具体的应用场景和需求来选择合适的输入图像大小。



### 练习 7.2.4

请参考VGG论文中的表1构建其他常见模型，如VGG-16或VGG-19。

**解答：** 

参考文献：https://arxiv.org/pdf/1409.1556.pdf

In [7]:
import torch
import torch.nn as nn
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## 7.3 网络中的网络（NiN）

### 练习 7.3.1

调整NiN的超参数，以提高分类准确性。

**解答：** 

可以将学习率调低一点，比如0.05，效果会好一些。

### 练习 7.3.2

为什么NiN块中有两个$1\times 1$卷积层？删除其中一个，然后观察和分析实验现象。

**解答：** 

第一个1x1卷积层用于将每个通道内的特征进行组合，将不同通道之间的特征进行混合，从而得到更加复杂的特征。第二个1x1卷积层则用于减少特征图的维度，将高维特征图进行压缩，减小模型的计算量。

删掉一个1×1卷积层后，准确率变高了。推测是当前的数据不需要对高维特征图再次压缩。原始NiN网络使用的是ImageNet数据，而我们用的是fashion_mnist数据集。



### 练习 7.3.3

计算NiN的资源使用情况。
1. 参数的数量是多少？
1. 计算量是多少？
1. 训练期间需要多少显存？
1. 预测期间需要多少显存？

**解答：** 

1. 参数量：(kernel * kernel) * channel_input * channel_output

  对于每个卷积层，参数数量为输入通道数乘以卷积核尺寸乘以输出通道数，加上输出通道数作为偏置项的数量。

  第一个nin_block的参数数量为11x11x3x96+1x1x96x96+1x1x96x96=53280

  第二个nin_block的参数数量为5x5x96x256+1x1x256x256+1x1x256x256=745472

  第三个nin_block的参数数量为3x3x256x384+1x1x384x384+1x1x384x384=1179648

  最后一个nin_block的参数数量为3x3x384x10+1x1x10x10+1x1x10x10=34760

  模型的总参数数量为53280+745472+1179648+34760=2013160

2. 计算量：(kernel* kernel* map* map) * channel_input * channel_output

  所以计算量等于当前层的参数量*map*map，即总计算量为53280*54*54+745472*27*27+1179648*13*13+34760*1*1=898208840≈8.9*10^8

3. 这个跟显卡的型号相关。

4. 这个跟显卡的型号相关。



### 练习 7.3.4

一次性直接将$384 \times 5 \times 5$的表示缩减为$10 \times 5 \times 5$的表示，会存在哪些问题？

**解答：** 

会导致信息严重丢失，可能会影响模型的准确性。

## 7.4 含并行链接的网络（GoogleLeNet） 

### 练习 7.4.1

GoogLeNet有一些后续版本。尝试实现并运行它们，然后观察实验结果。这些后续版本包括：
1. 添加批量规范化层 （batch normalization），（在7.5节中将介绍）；
1. 对Inception模块进行调整；
1. 使用标签平滑（label smoothing）进行模型正则化；
1. 加入残差连接（将在7.6节中介绍）。

**解答：** 

In [8]:
class Inception(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        self.p1_bn = nn.BatchNorm2d(c1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_bn_1 = nn.BatchNorm2d(c2[0])
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        self.p2_bn_2 = nn.BatchNorm2d(c2[1])
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_bn_1 = nn.BatchNorm2d(c3[0])
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        self.p3_bn_2 = nn.BatchNorm2d(c3[1])
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)
        self.p4_bn = nn.BatchNorm2d(c4)

    def forward(self, x):
        p1 = F.relu(self.p1_bn(self.p1_1(x)))
        p2 = F.relu(self.p2_bn_2(self.p2_2(F.relu(self.p2_bn_1(self.p2_1(x))))))
        p3 = F.relu(self.p3_bn_2(self.p3_2(F.relu(self.p3_bn_1(self.p3_1(x))))))
        p4 = F.relu(self.p4_bn(self.p4_2(self.p4_1(x))))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3, p4), dim=1)

### 练习 7.4.2 

使用GoogLeNet的最小图像大小是多少？

**解答：** 

224 * 224

### 练习 7.4.3

将AlexNet、VGG和NiN的模型参数大小与GoogLeNet进行比较。后两个网络架构是如何显著减少模型参数大小的？

**解答：** 

NiN采用了全局平均池化，可以将卷积层的输出直接平均成一个数，从而减少了全连接层的参数数量。NiN还采用了1x1卷积，可以在不改变特征图大小的情况下改变特征图的深度，从而进一步减少了模型参数数量。

GoogLeNet使用了Inception块，可以通过并联的方式将不同的卷积层结合在一起，从而减少模型参数大小。



## 7.5 批量规范化 

### 练习 7.5.1 

在使用批量规范化之前，我们是否可以从全连接层或卷积层中删除偏置参数？为什么？

**解答：** 

可以。BatchNorm的作用是对每个batch的数据进行规范化，使得每个batch的数据分布相同，从而加速模型训练。因此，BatchNorm可以替代全连接层或卷积层中的偏置参数，因为它可以自动地学习偏置参数。



### 练习 7.5.2

比较LeNet在使用和不使用批量规范化情况下的学习率。
1. 绘制训练和测试准确度的提高。
1. 学习率有多高？

**解答：** 

1. train acc 0.911, test acc 0.884
1. lr = 1

### 练习 7.5.3

我们是否需要在每个层中进行批量规范化？尝试一下？

**解答：** 

除了输出层都可以加入`BatchNorm`，因为加了其实没什么作用

全连接层和卷积层输出上，激活函数前

全连接层和卷积层输入上



### 练习 7.5.4

可以通过批量规范化来替换暂退法吗？行为会如何改变？

**解答：** 

可以，但一般是直接进行替换，不会同时用，因为二者都是起到正则项的作用

### 练习 7.5.5

确定参数`beta`和`gamma`，并观察和分析结果。

**解答：** 

### 练习 7.5.6

查看高级API中有关`BatchNorm`的在线文档，以查看其他批量规范化的应用。

**解答：** 

[BatchNorm1d](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html?highlight=batchnorm#torch.nn.BatchNorm1d)

[BatchNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html?highlight=batchnorm#torch.nn.BatchNorm2d)


### 练习 7.5.7

研究思路：可以应用的其他“规范化”转换？可以应用概率积分变换吗？全秩协方差估计可以么？

**解答：** 

## 7.6 残差网络（ResNet） 

### 练习 7.6.1

图7-5中的Inception块与残差块之间的主要区别是什么？在删除了Inception块中的一些路径之后，它们是如何相互关联的？

**解答：** 

它们的主要区别在于，Inception块是由多个不同大小的卷积核组成的，可以在同一层上获得稀疏或非稀疏的特征；而残差块则是通过添加跨层连接来解决深度神经网络中的梯度消失问题。

当删除了Inception块中的一些路径之后，它们之间的关系会发生变化，因为这些路径是相互关联的。

### 练习 7.6.2

参考ResNet论文中的表1，以实现不同的变体。

**解答：** 

### 练习 7.6.3

对于更深层次的网络，ResNet引入了“bottleneck”架构来降低模型复杂性。请试着去实现它。

**解答：** 

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

### 练习 7.6.4

在ResNet的后续版本中，作者将“卷积层、批量规范化层和激活层”架构更改为“批量规范化层、激活层和卷积层”架构。请尝试做这个改进。详见参考文献[57]中的图1。

**解答：** 

### 练习 7.6.5

为什么即使函数类是嵌套的，我们仍然要限制增加函数的复杂性呢？

**解答：** 

因为过于复杂的函数会导致过拟合，从而降低模型的泛化能力。

## 7.7 稠密连接网络（DenseNet）

### 练习 7.7.1

为什么我们在过渡层使用平均汇聚层而不是最大汇聚层？

**解答：** 

因为平均汇聚层可以更好地保留特征图中的信息，而最大汇聚层会丢失一些信息。

### 练习 7.7.2

DenseNet的优点之一是其模型参数比ResNet小。为什么呢？

**解答：**

因为DenseNet中每个卷积层的输入都是前面所有层的输出的拼接，而不是像ResNet一样只是前面一层的输出。这种设计使得DenseNet中每个卷积层的输入通道数比ResNet少很多，因此DenseNet中的BN层参数也会少很多，全连接层的参数也比ResNet少很多。

### 练习 7.7.3

DenseNet一个诟病的问题是内存或显存消耗过多。
1. 真的是这样吗？可以把输入形状换成$224 \times 224$，来看看实际的显存消耗。
1. 有另一种方法来减少显存消耗吗？需要改变框架么？

**解答：** 

1. 略
1. 使用更小的batch size，这样可以减少显存的使用;
   使用更小的图像尺寸，这样可以减少每个卷积层的输入和输出的大小;
   使用更小的模型，这样可以减少模型中参数的数量;

### 练习 7.7.4

实现DenseNet论文[71]表1所示的不同DenseNet版本。

**解答：** 

### 练习 7.7.5

应用DenseNet的思想设计一个基于多层感知机的模型。将其应用于4.10节中的房价预测任务。

**解答：** 